XGboost practice

In [42]:
import xgboost as xgb
import pandas as pd


In [48]:
train = pd.read_csv('/Users/swenson/Downloads/all/train.csv',encoding='big5')
test = pd.read_csv('/Users/swenson/Downloads/all/test.csv',encoding='big5')

In [7]:
# Taking a look at how many rows and columns the train dataset contains
rows = train.shape[0]
columns = train.shape[1]
print("The train dataset contains {0} rows and {1} columns".format(rows, columns))

# any() applied twice to check run the isnull check across all columns.
train.isnull().any().any()

The train dataset contains 1460 rows and 81 columns


True

In [11]:
#查看資料
train.head()
#每欄Null的個數
1460-train.count()   

Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
                 ... 
BedroomAbvGr        0
KitchenAbvGr        0
KitchenQual         0
TotRmsAbvGrd        0
Functional          0
Fireplaces          0
FireplaceQu       690
GarageType         81
GarageYrBlt        81
GarageFinish       81
GarageCars          0
GarageArea          0
GarageQual         81
GarageCond         81
PavedDrive

In [49]:
#先只用數值變數
train = train.select_dtypes(include= ['number'])
test = test.select_dtypes(include= ['number'])
print(train.head())
#刪ID
train = train.drop(['Id'],axis=1)

   Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0     8450            7            5       2003   
1   2          20         80.0     9600            6            8       1976   
2   3          60         68.0    11250            7            5       2001   
3   4          70         60.0     9550            7            5       1915   
4   5          60         84.0    14260            8            5       2000   

   YearRemodAdd  MasVnrArea  BsmtFinSF1    ...      WoodDeckSF  OpenPorchSF  \
0          2003       196.0         706    ...               0           61   
1          1976         0.0         978    ...             298            0   
2          2002       162.0         486    ...               0           42   
3          1970         0.0         216    ...               0           35   
4          2000       350.0         655    ...             192           84   

   EnclosedPorch  3SsnPorch  ScreenPorch  Po

In [38]:
#把NA全變成平均
train=train.fillna(train.mean())
train.count()

Id               1460
MSSubClass       1460
LotFrontage      1460
LotArea          1460
OverallQual      1460
OverallCond      1460
YearBuilt        1460
YearRemodAdd     1460
MasVnrArea       1460
BsmtFinSF1       1460
BsmtFinSF2       1460
BsmtUnfSF        1460
TotalBsmtSF      1460
1stFlrSF         1460
2ndFlrSF         1460
LowQualFinSF     1460
GrLivArea        1460
BsmtFullBath     1460
BsmtHalfBath     1460
FullBath         1460
HalfBath         1460
BedroomAbvGr     1460
KitchenAbvGr     1460
TotRmsAbvGrd     1460
Fireplaces       1460
GarageYrBlt      1460
GarageCars       1460
GarageArea       1460
WoodDeckSF       1460
OpenPorchSF      1460
EnclosedPorch    1460
3SsnPorch        1460
ScreenPorch      1460
PoolArea         1460
MiscVal          1460
MoSold           1460
YrSold           1460
SalePrice        1460
dtype: int64

In [35]:
##先Try 看看迴歸
from sklearn.linear_model import LinearRegression
from sklearn import cross_validation, tree, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score

regr = linear_model.LinearRegression()

In [36]:
#把資料集拆成 X and  Y
train_Y = train['SalePrice']
train_x = train.drop(['SalePrice'],axis=1)

In [37]:
#sample train and vali data
X_train, X_test, y_train, y_test = cross_validation.train_test_split(train_x, train_Y ,test_size=0.2)

In [26]:
#Train model!
regr.fit(X_train, y_train)
print(regr.predict(X_test))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [40]:
import numpy as np
print(np.any(np.isnan(train)))

False


In [45]:
## 開始XgBoost
xgb = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,colsample_bytree=1, max_depth=7)
xgb.fit(X_train,y_train)
predictions = xgb.predict(test)
print(explained_variance_score(predictions,y_test))

0.829803439279


In [52]:
# 預測Test data
Y_pred = xgb.predict(test)

In [54]:
#製作上傳資料
submission = pd.DataFrame({
        "Id": test["Id"],
        "SalePrice": Y_pred
    })
    
submission.to_csv('/Users/swenson/Downloads/all/submission.csv',index=False)


Result<br />
https://www.kaggle.com/c/house-prices-advanced-regression-techniques/leaderboard<br />
score = 0.14476<br />
rank = 2348